##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Composición de algoritmos de aprendizaje

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/federated/tutorials/composing_learning_algorithms"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/federated/tutorials/composing_learning_algorithms.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/federated/tutorials/composing_learning_algorithms.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/federated/tutorials/composing_learning_algorithms.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a>
</td>
</table>

## Antes de empezar

Antes de empezar, ejecute lo que se encuentra a continuación, para asegurarse de que el entorno esté preparado correctamente. Si no ve un mensaje de inicio, para más instrucciones, consulte la guía de [instalación](../install.md). 

In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated

In [ ]:
from collections.abc import Callable

import tensorflow as tf
import tensorflow_federated as tff

**NOTA**: Esta colaboración ha sido verificada para trabajar con la [versión de lanzamiento más reciente](https://github.com/tensorflow/federated#compatibility) del paquete pip `tensorflow_federated`, pero el proyecto federado de TensorFlow aún se encuentra en una etapa de desarrollo previa al lanzamiento. Por lo tanto, es probable que no funcione en `main`.

# Composición de algoritmos de aprendizaje

En el [tutorial sobre cómo crear un algoritmo propio de aprendizaje federado](https://github.com/tensorflow/federated/blob/v0.62.0/docs/tutorials/building_your_own_federated_learning_algorithm.ipynb) se usó el núcleo federado de TFF para implementar directamente una versión del algoritmo de promedio federado (FedAvg).

En este tutorial, usaremos los componentes del aprendizaje federado en la API de TFF para crear algoritmos de aprendizaje federado de forma modular, sin tener que volver a implementar todo de cero.

Para este tutorial, implementaremos una variante de FedAvg que emplea recortes (<em>clipping</em>) mediante el entrenamiento local.

## Bloques para construcción de algoritmos de aprendizaje

A un alto nivel, muchos algoritmos de aprendizaje se pueden separar en 4 componentes, a los que denominamos **bloques de construcción**. Son los siguientes:

1. Distribuidor (las comunicaciones del servidor al cliente)
2. Trabajo del cliente (el cálculo del cliente local)
3. Agregador (las comunicaciones del cliente al servidor)
4. Finalizador (el cálculo del servidor usando salidas de cliente agregadas)

Si bien es cierto que en el [tutorial sobre cómo crear un algoritmo propio de aprendizaje federado](https://github.com/tensorflow/federated/blob/v0.62.0/docs/tutorials/building_your_own_federated_learning_algorithm.ipynb) se implementaron todos estos bloques de construcción partiendo de cero, por lo general, no es necesario hacerlo de este modo. En cambio, podemos reutilizar los bloques de construcción de algoritmos similares.

En este caso, para implementar FedAvg con recorte (<em>clipping</em>) de gradiente, solamente hace falta modificar el bloque de construcción del **trabajo del cliente**. El resto de los bloques puede ser idéntico a lo que se usa para FedAvg "vainilla".

# Implementación del trabajo del cliente

Primero, escribamos la lógica TF que lleva a cabo el entrenamiento del modelo local con recorte de gradiente. Para simplificar, los gradientes que se recortarán tienen una norma mayormente de 1.

## Lógica de TF

In [ ]:
@tf.function
def client_update(model: tff.learning.models.VariableModel,
                  dataset: tf.data.Dataset,
                  server_weights: tff.learning.models.ModelWeights,
                  client_optimizer: tf.keras.optimizers.Optimizer):
  """Performs training (using the server model weights) on the client's dataset."""
  # Initialize the client model with the current server weights.
  client_weights = tff.learning.models.ModelWeights.from_model(model)
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        client_weights, server_weights)

  # Use the client_optimizer to update the local model.
  # Keep track of the number of examples as well.
  num_examples = 0.0
  for batch in dataset:
    with tf.GradientTape() as tape:
      # Compute a forward pass on the batch of data
      outputs = model.forward_pass(batch)
      num_examples += tf.cast(outputs.num_examples, tf.float32)

    # Compute the corresponding gradient
    grads = tape.gradient(outputs.loss, client_weights.trainable)

    # Compute the gradient norm and clip
    gradient_norm = tf.linalg.global_norm(grads)
    if gradient_norm > 1:
      grads = tf.nest.map_structure(lambda x: x/gradient_norm, grads)

    grads_and_vars = zip(grads, client_weights.trainable)

    # Apply the gradient using a client optimizer.
    client_optimizer.apply_gradients(grads_and_vars)

  # Compute the difference between the server weights and the client weights
  client_update = tf.nest.map_structure(tf.subtract,
                                        client_weights.trainable,
                                        server_weights.trainable)

  return tff.learning.templates.ClientResult(
      update=client_update, update_weight=num_examples)

Hay algunos puntos importantes sobre el código que aquí figura (arriba). Primero, da seguimiento a la cantidad de ejemplos vistos, ya que será lo que constituya el *peso* de la actualización del cliente (cuando calculamos un promedio de los clientes).

En segundo lugar, usa [`tff.learning.templates.ClientResult`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/templates/ClientResult) para empacar la salida. Este tipo de retorno se utiliza para estandarizar los bloques de construcción del trabajo del cliente en `tff.learning`.

## Creación de un ClientWorkProcess

Mientras la lógica de TF anterior hará el entrenamiento local con recortes, todavía deberá encapsularse (<em>wrap</em>) en código de TFF para crear el bloque de construcción necesario.

Específicamente, los 4 bloques de construcción están representados en forma de un [`tff.templates.MeasuredProcess`](https://www.tensorflow.org/federated/api_docs/python/tff/templates/MeasuredProcess). Significa que los 4 bloques tienen tanto una función `initialize` como una `next`, que se utilizan para instanciar y ejecutar el cálculo.

Esto permite que cada bloque de construcción controle su propio **estado** (almacenado en el servidor) según le sea necesario para realizar sus propias operaciones. Si bien no se utilizará en este tutorial, sí se puede usar para cosas como el seguimiento de la cantidad de operaciones que se hayan producido, o para controlar los estados del optimizador.

La lógica de TF sobre el trabajo del cliente, por lo general, debería encapsularse como un [`tff.learning.templates.ClientWorkProcess`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/templates/ClientWorkProcess), que codifica los tipos esperados que entran y salen del entrenamiento local del cliente. Se puede parametrizar mediante un modelo y el optimizador, tal como se muestra a continuación.

In [ ]:
def build_gradient_clipping_client_work(
    model_fn: Callable[[], tff.learning.models.VariableModel],
    optimizer_fn: Callable[[], tf.keras.optimizers.Optimizer],
) -> tff.learning.templates.ClientWorkProcess:
  """Creates a client work process that uses gradient clipping."""

  with tf.Graph().as_default():
    # Wrap model construction in a graph to avoid polluting the global context
    # with variables created for this model.
    model = model_fn()
  data_type = tff.SequenceType(model.input_spec)
  model_weights_type = tff.learning.models.weights_type_from_model(model)

  @tff.federated_computation
  def initialize_fn():
    return tff.federated_value((), tff.SERVER)

  @tff.tf_computation(model_weights_type, data_type)
  def client_update_computation(model_weights, dataset):
    model = model_fn()
    optimizer = optimizer_fn()
    return client_update(model, dataset, model_weights, optimizer)

  @tff.federated_computation(
      initialize_fn.type_signature.result,
      tff.type_at_clients(model_weights_type),
      tff.type_at_clients(data_type)
  )
  def next_fn(state, model_weights, client_dataset):
    client_result = tff.federated_map(
        client_update_computation, (model_weights, client_dataset))
    # Return empty measurements, though a more complete algorithm might
    # measure something here.
    measurements = tff.federated_value((), tff.SERVER)
    return tff.templates.MeasuredProcessOutput(state, client_result,
                                               measurements)
  return tff.learning.templates.ClientWorkProcess(
      initialize_fn, next_fn)

# Composición de un algoritmo de aprendizaje

Pongamos el trabajo del cliente de arriba en un algoritmo completo y funcional. Primero, configuremos nuestros datos y el modelo.

## Preparación de los datos de entrada

Cargue y procese el conjunto de datos EMNIST incluido en TFF. Para más detalles, consulte el tutorial sobre [clasificación de imágenes](federated_learning_for_image_classification.ipynb).

In [ ]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

A fin de alimentar nuestro modelo con el conjunto de datos, estos datos se aplanan y se transforman en tuplas con la forma `(flattened_image_vector, label)`.

Ahora, seleccione una pequeña cantidad de clientes y aplique el preprocesamiento anterior a los conjuntos de datos.

In [ ]:
NUM_CLIENTS = 10
BATCH_SIZE = 20

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch of EMNIST data and return a (features, label) tuple."""
    return (tf.reshape(element['pixels'], [-1, 784]), 
            tf.reshape(element['label'], [-1, 1]))

  return dataset.batch(BATCH_SIZE).map(batch_format_fn)

client_ids = sorted(emnist_train.client_ids)[:NUM_CLIENTS]
federated_train_data = [preprocess(emnist_train.create_tf_dataset_for_client(x))
  for x in client_ids
]

## Preparación del modelo

Se usa el mismo modelo que se utilizó en el tutorial sobre [clasificación de imágenes](federated_learning_for_image_classification.ipynb). Este modelo (implementado mediante `tf.keras`) tiene una capa oculta seguida por una capa softmax. Para usar este modelo en TFF, el modelo Keras se encapsula como un [`tff.learning.models.VariableModel`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model). Esto permite realizar el [pase hacia adelante](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model#forward_pass) del modelo dentro de TFF y [extraer las salidas del modelo](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model#report_local_unfinalized_metrics). Para más detalles, también consulte el tutorial sobre [clasificación de imágenes](federated_learning_for_image_classification.ipynb).

In [ ]:
def create_keras_model():
  initializer = tf.keras.initializers.GlorotNormal(seed=0)
  return tf.keras.models.Sequential([
      tf.keras.layers.Input(shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer=initializer),
      tf.keras.layers.Softmax(),
  ])

def model_fn():
  keras_model = create_keras_model()
  return tff.learning.models.from_keras_model(
      keras_model,
      input_spec=federated_train_data[0].element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

## Preparación de los optimizadores

Al igual que en [`tff.learning.algorithms.build_weighted_fed_avg`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_avg), aquí hay dos optimizadores: uno del cliente y otro del servidor. Para hacerlo más simple, los optimizadores serán de SGD (Stochastic Gradient Descent, descenso de gradiente estocástico) con diferentes tasas de aprendizaje.

In [ ]:
client_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate=0.01)
server_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate=1.0)

## Definición de los bloques de construcción

Ahora que ya hemos configurado el bloque de construcción del trabajo del cliente, los datos, el modelo y los optimizadores, queda pendiente la creación de los bloques de construcción para el distribuidor, el agregador y el finalizador. Se pueden hacer tomando prestados algunos predeterminados de TFF que utiliza FedAvg.

In [ ]:
@tff.tf_computation()
def initial_model_weights_fn():
  return tff.learning.models.ModelWeights.from_model(model_fn())

model_weights_type = initial_model_weights_fn.type_signature.result

distributor = tff.learning.templates.build_broadcast_process(model_weights_type)
client_work = build_gradient_clipping_client_work(model_fn, client_optimizer_fn)

# TFF aggregators use a factory pattern, which create an aggregator
# based on the output type of the client work. This also uses a float (the number
# of examples) to govern the weight in the average being computed.)
aggregator_factory = tff.aggregators.MeanFactory()
aggregator = aggregator_factory.create(model_weights_type.trainable,
                                       tff.TensorType(tf.float32))
finalizer = tff.learning.templates.build_apply_optimizer_finalizer(
    server_optimizer_fn, model_weights_type)

## Composición de los bloques de construcción

Finalmente, para reunir los bloques de construcción, podemos usar un **compositor** integrado en TFF. Se trata de un compositor simple que toma los 4 bloques de construcción de arriba y conecta sus tipos.

In [ ]:
fed_avg_with_clipping = tff.learning.templates.compose_learning_process(
    initial_model_weights_fn,
    distributor,
    client_work,
    aggregator,
    finalizer
)

# Ejecución del algoritmo

Ahora que el algoritmo está terminado, ejecutémoslo. Primero, **inicialicemos** el algoritmo. El **estado** de este algoritmo tiene un componente para cada bloque de construcción, junto con otro para los *pesos del modelo global*.

In [ ]:
state = fed_avg_with_clipping.initialize()

state.client_work

()

Tal como es de esperar, el trabajo del cliente tiene un estado vacío (recuerde el código de trabajo del cliente que figura arriba). Por ejemplo, el finalizador controla cuántas iteraciones se han producido. Como `next` todavía no se ha ejecutado, tiene un estado de `0`.

In [ ]:
state.finalizer

[0]

Ahora, ejecutemos una ronda de entrenamiento.

In [ ]:
learning_process_output = fed_avg_with_clipping.next(state, federated_train_data)

La salida en este caso (`tff.learning.templates.LearningProcessOutput`) tiene una salida de `.state` y una de `.metrics`. Observemos ambas.

In [ ]:
learning_process_output.state.finalizer

[1]

Claramente, el estado del finalizador ha aumentado en uno, ya que se ha ejecutado la ronda uno de `.next`.

In [ ]:
learning_process_output.metrics

OrderedDict([('distributor', ()),
             ('client_work', ()),
             ('aggregator',
              OrderedDict([('mean_value', ()), ('mean_weight', ())])),
             ('finalizer', ())])

Si bien es cierto que las métricas están vacías, para algoritmos más complejos y prácticos, por lo general, estarán llenas de información útil.

# Conclusión

Al aplicar el marco de trabajo de bloques de construcción o de compositores que utilizamos aquí, podemos crear algoritmos nuevos de aprendizaje enteros, sin tener que rehacer todo desde cero. Sin embargo, este solamente es el punto de partida. Esta metodología de trabajo facilita muchísimo la expresión de algoritmos como modificaciones simples de FedAvg. Para más información sobre los algoritmos, consulte [`tff.learning.algorithms`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms), que contiene algoritmos como [FedProx](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_prox) y [FedAvg con programación de tasa de aprendizaje de clientes](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_avg_with_optimizer_schedule). Estas API pueden colaborar con la implementación de algoritmos totalmente nuevos, como [la agrupación en clústeres de k-medias federadas](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_fed_kmeans).